In [10]:
!pip3 install --upgrade alexnet_pytorch

Requirement already up-to-date: alexnet_pytorch in /home/samhitha/.local/lib/python3.8/site-packages (0.2.0)


In [11]:
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from alexnet_pytorch import AlexNet
import torch.optim as optim

In [12]:
torch.manual_seed(0)

In [20]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize(224),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.EuroSAT(root="./Data1/train",transform=transform,download=True)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=8,shuffle=True, num_workers=2)

testset = torchvision.datasets.EuroSAT(root='./Data1/test',download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=8,shuffle=False, num_workers=2)

classes = ('AnnualCrop', 'Forest', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake', 'HerbaceousVegetation')

In [21]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

Using cache found in /home/samhitha/.cache/torch/hub/pytorch_vision_v0.6.0


In [22]:
AlexNet_model.classifier[4] = nn.Linear(4096,1024)
AlexNet_model.classifier[6] = nn.Linear(1024,10)

In [23]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(AlexNet_model.parameters(), lr=0.001,momentum=0.9)

In [24]:
start_time = time.time()
for iteration in range(10):  # loop over the dataset multiple times

    iterative_loss = 0.0
    for i, (inputs,labels) in enumerate(trainloader, 0):

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = AlexNet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        iterative_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (iteration + 1, i + 1, iterative_loss / 1000))
            # print(loss.item())
            iterative_loss = 0.0

print('Finished Training')
print("Time taken to train is %s seconds" % (time.time() - start_time))

[1,  1000] loss: 0.851
[1,  2000] loss: 0.513
[1,  3000] loss: 0.430
[2,  1000] loss: 0.326
[2,  2000] loss: 0.295
[2,  3000] loss: 0.287
[3,  1000] loss: 0.227
[3,  2000] loss: 0.251
[3,  3000] loss: 0.213
[4,  1000] loss: 0.174
[4,  2000] loss: 0.183
[4,  3000] loss: 0.162
[5,  1000] loss: 0.151
[5,  2000] loss: 0.141
[5,  3000] loss: 0.147
[6,  1000] loss: 0.113
[6,  2000] loss: 0.117
[6,  3000] loss: 0.114
[7,  1000] loss: 0.095
[7,  2000] loss: 0.110
[7,  3000] loss: 0.099
[8,  1000] loss: 0.087
[8,  2000] loss: 0.089
[8,  3000] loss: 0.099
[9,  1000] loss: 0.071
[9,  2000] loss: 0.061
[9,  3000] loss: 0.074
[10,  1000] loss: 0.070
[10,  2000] loss: 0.084
[10,  3000] loss: 0.080
Finished Training
Time taken to train is 18644.870888471603 seconds


In [25]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = AlexNet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 97 %


In [27]:
#Instantiating CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [28]:
#Testing classification accuracy for individual classes.
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = AlexNet_model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of AnnualCrop : 99 %
Accuracy of Forest : 95 %
Accuracy of Highway : 94 %
Accuracy of Industrial : 98 %
Accuracy of Pasture : 99 %
Accuracy of PermanentCrop : 98 %
Accuracy of Residential : 97 %
Accuracy of River : 99 %
Accuracy of SeaLake : 96 %
Accuracy of HerbaceousVegetation : 99 %
